In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data

import random
from tqdm import tqdm_notebook as tqdm
random.seed(42)

In [2]:
import random


In [3]:
class DataLoader(data.Dataset):
    def __init__(self,max_train_value=1000, max_n=100,training_set_size=1000):
        random_a_b_s=torch.sort(torch.randint(low=1,high=max_train_value+1,size=(training_set_size,2)))[0].double()
        random_n_s=torch.randint(low=3,high=max_n, size=(training_set_size,1)).double()
        self.lis_expected_outputs=[]
        self.lis_inputs=[]
        for index in range(training_set_size):
            temp_input=[0,0,0]

            temp_input[0]=random_a_b_s[index][0]
            temp_input[1]=random_a_b_s[index][1]
            temp_input[2]=random_n_s[index]
            expected_output=[temp_input[0],temp_input[1]]
            for i in range(int(temp_input[2])-2):
                expected_output.append(expected_output[i]+expected_output[i+1])
            input=torch.tensor(temp_input, dtype=torch.float64).unsqueeze(0)

            output=torch.tensor(expected_output, dtype=torch.float64)
            self.lis_expected_outputs.append(output)
            self.lis_inputs.append(input)
    def __len__(self):
        return len(self.lis_inputs)
    def __getitem__(self, idx):
        dic_in_out={'input':self.lis_inputs[idx] , 'output':self.lis_expected_outputs[idx]}
        return dic_in_out

In [4]:


class MODEL(nn.Module):
    def __init__(self , input_size ,hidden_size,  output_size):
        super(MODEL, self).__init__()
        self.hidden_size=hidden_size
        self.i2h=nn.Linear(input_size+hidden_size, hidden_size,bias=False).double()
        self.i2o=nn.Linear(input_size+hidden_size, output_size,bias=False).double()


    def step(self, input , hidden):
        combined=torch.cat((input,hidden),1)
        hidden=self.i2h(combined)
        output=self.i2o(combined)
        return output,hidden
    def forward(self , input):
        hidden=self.initHidden()
        n=int(input[0][2])
        a=torch.tensor([int(input[0][0])],dtype=torch.float64).unsqueeze(0)
        b=torch.tensor([int(input[0][1])] ,dtype=torch.float64).unsqueeze(0)

        predictions=torch.zeros(size=(1,n)).double()
        predictions[0,0]=a
        predictions[0,1]=b
        _,hidden=self.step(torch.tensor([a[0,0]], dtype=torch.float64).clone().unsqueeze(0),hidden)
        for i in range(n-2):
            temp_output,hidden=self.step(torch.tensor([predictions[0,i+1]] , dtype=torch.float64).clone().unsqueeze(0) , hidden)
            predictions[0,i+2]=temp_output[0]
        return predictions
    def initHidden(self):
        return torch.zeros(1, self.hidden_size).double()


In [5]:

n_hidden=1
n_input=1
n_output=1

model=MODEL(n_input, n_hidden , n_output)

In [6]:
def RelativeMSE(output, target):
    return torch.mean(torch.mul(1/target , (target-output))**2)

In [7]:
criterion=RelativeMSE

In [17]:
learning_rate=1e-2
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate )
def train(epochs=30):
    dataset=DataLoader()
    for i in tqdm(range(epochs), desc = 'epochs'):
        for idx in tqdm(range(len(dataset)) , desc='sample'):
            sample=dataset[idx]
            input=sample['input']
            expected_output=sample['output']
            predictions=model(input )
            loss=criterion(predictions , expected_output)

            optimizer.zero_grad()

            loss.backward()

            clip=100
            nn.utils.clip_grad_norm_(model.parameters() , clip)
            optimizer.step()
        

    


In [22]:
train()

In [23]:

output=(model(torch.tensor([13,40,5], dtype=torch.float64).unsqueeze(0)))
print(output)


tensor([[ 13.0000,  40.0000,  53.3243,  93.4055, 146.7079]],
       dtype=torch.float64, grad_fn=<CopySlices>)


In [24]:
print(model.i2o.weight.data)
print(model.i2h.weight.data)

tensor([[1.0102, 0.9180]], dtype=torch.float64)
tensor([[ 1.0824, -0.0160]], dtype=torch.float64)


In [25]:
#save the maodel to a data file
torch.save(model, "./data.pt")